# Ingest NCEP GFS 0.25 Degree Data for 6 hour forecasts. 

#### 1.) Conda package installations to environment and importing appropriate libraries. 

In [1]:
# conda install -c conda-forge gdal
# conda install -c conda-forge geopandas
# conda install -c conda-forge earthpy
# conda install -c conda-forge cloudpathlib
# conda install -c conda-forge pyhdf
# conda install -c anaconda basemap

#conda install -c conda-forge xarray
#conda install -c conda-forge ipywidgets
#conda install -c conda-forge cartopy
## For IO dependencies in xarray 
#conda install -c conda-forge xarray dask netCDF4 bottleneck
#conda install -c conda-forge cfgrib
#conda install -c conda-forge pygrib
#conda install -c yt87 pywgrib2_xr

In [1]:
#Make sure we are in right conda env. 
!jupyter kernelspec list

Available kernels:
  python3    /home/ec2-user/.conda/envs/capstone/share/jupyter/kernels/python3


In [74]:
#Import Packages. 
import sys
import os
import requests
import warnings
import glob

import matplotlib.pyplot as plt
import seaborn as sns
import numpy.ma as ma
import numpy as np
#from shapely.geometry import mapping, box
import geopandas as gpd
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
from osgeo import gdal
import pandas as pd

#GFS data
import xarray # used for reading the data.
import xarray_extras.csv # used for writing data to csv format. 
import pygrib
import xarray # used for reading the data.
import ipywidgets as widgets
import matplotlib.pyplot as plt # used to plot the data.
import ipywidgets as widgets # For ease in selecting variables.
import cartopy.crs as ccrs # Used to georeference data.


# #from cloudpathlib import S3Path, S3Client
# from pyhdf.SD import SD, SDC

warnings.simplefilter('ignore')

#### 2.) Download data from NCAR servers. 

In [3]:
 ## First, we need to authenticate
try:
    import getpass
    input = getpass.getpass
except:
    try:
        input = raw_input
    except:
        pass

In [4]:
## Now, we need your password.
pswd = input('password: ')

password:  ···········


In [5]:
values = {'email' : 'jericojohns@berkeley.edu', 'passwd' : pswd, 'action' : 'login'}
login_url = 'https://rda.ucar.edu/cgi-bin/login'

In [6]:
ret = requests.post(login_url, data=values)
if ret.status_code != 200:
    print('Bad Authentication')
    print(ret.text)
    exit(1)

In [7]:
dspath = 'https://rda.ucar.edu/data/ds084.1/'
filelist = ['2018/20180101/gfs.0p25.2018010100.f006.grib2']

In [8]:
save_dir = '/local/train/GFS/'

 #### Now to download the files

In [9]:
for file in filelist:
    filename = dspath + file
    outfile = save_dir + os.path.basename(filename)
    print('Downloading', file)
    req = requests.get(filename, cookies = ret.cookies, allow_redirects=True)
    open(outfile, 'wb').write(req.content)

#### Once you have downloaded the data, the next part can help you plot it.

In [11]:
filelist_arr = [save_dir + os.path.basename(file) for file in filelist]
selected_file = widgets.Dropdown(options=filelist_arr, description='data file')
display(selected_file)

Dropdown(description='data file', options=('/local/train/GFS/gfs.0p25.2018010100.f006.grib2',), value='/local/…

In [90]:
# Now to load in the data to xarray
type_of_level1 = 'surface' # for Temperature and Planetary Boundary Layer Height
type_of_level2 = 'atmosphereSingleLayer' # for Relative Humidity
ds_level_surface = xarray.open_dataset(selected_file.value, filter_by_keys={'typeOfLevel': type_of_level1}, engine="cfgrib")
ds_level_atmosphere = xarray.open_dataset(selected_file.value, filter_by_keys={'typeOfLevel': type_of_level2}, engine="cfgrib")

Ignoring index file '/local/train/GFS/gfs.0p25.2018010100.f006.grib2.923a8.idx' older than GRIB file
Ignoring index file '/local/train/GFS/gfs.0p25.2018010100.f006.grib2.923a8.idx' older than GRIB file


In [91]:
ds_level_surface

<xarray.Dataset>
Dimensions:     (latitude: 721, longitude: 1440)
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    valid_time  datetime64[ns] ...
Data variables: (12/42)
    vis         (latitude, longitude) float32 ...
    gust        (latitude, longitude) float32 ...
    hindex      (latitude, longitude) float32 ...
    sp          (latitude, longitude) float32 ...
    orog        (latitude, longitude) float32 ...
    t           (latitude, longitude) float32 ...
    ...          ...
    4lftx       (latitude, longitude) float32 ...
    hpbl        (latitude, longitude) float32 ...
    lsm         (latitude, longitude) float32 ...
    siconc      (latitude, longitude) float32 ...
    al          (latitude, longitude) float32 ...
    landn       (latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 
    history:                 2022-02-16T01:30 GRIB to CDM+CF via cfgrib-0.9.1...

In [92]:
ds_level_surface.info

<bound method Dataset.info of <xarray.Dataset>
Dimensions:     (latitude: 721, longitude: 1440)
Coordinates:
    time        datetime64[ns] 2018-01-01
    step        timedelta64[ns] 06:00:00
    surface     float64 0.0
  * latitude    (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    valid_time  datetime64[ns] 2018-01-01T06:00:00
Data variables: (12/42)
    vis         (latitude, longitude) float32 ...
    gust        (latitude, longitude) float32 ...
    hindex      (latitude, longitude) float32 ...
    sp          (latitude, longitude) float32 ...
    orog        (latitude, longitude) float32 ...
    t           (latitude, longitude) float32 ...
    ...          ...
    4lftx       (latitude, longitude) float32 ...
    hpbl        (latitude, longitude) float32 ...
    lsm         (latitude, longitude) float32 ...
    siconc      (latitude, longitude) float32 ...
    al          (latitude, l

In [15]:
#Extract valid time (time forecast is in effect. + 6 hours for 6 hour forecast).
#Instead of appending as a row in final csv we will name file using timeestamp. 
valid_time = ds.coords['valid_time'].values
valid_time_str = np.datetime_as_string(valid_time) 
valid_time_str

'2018-01-01T06:00:00.000000000'

In [36]:
ds2.variables['r'].values

array([[21., 21., 21., ..., 21., 21., 21.],
       [21., 21., 21., ..., 21., 21., 21.],
       [21., 21., 21., ..., 21., 21., 21.],
       ...,
       [ 2.,  2.,  2., ...,  2.,  2.,  2.],
       [ 2.,  2.,  2., ...,  2.,  2.,  2.],
       [ 2.,  2.,  2., ...,  2.,  2.,  2.]], dtype=float32)

In [164]:
#Define variable names
var_t = 't' #temperature (K) 
var_hpbl = 'hpbl' #Planetary Boundary Layer Height (m)
var_r = 'r' #Relative Humidity %

#Define filtered datasets (for each variable). 
ds_t = ds_level_surface[var_t] 
ds_hpbl = ds_level_surface[var_hpbl]
ds_r = ds_level_atmosphere[var_r]

In [94]:
ds_t

<xarray.DataArray 't' (latitude: 721, longitude: 1440)>
[1038240 values with dtype=float32]
Coordinates:
    time        datetime64[ns] 2018-01-01
    step        timedelta64[ns] 06:00:00
    surface     float64 0.0
  * latitude    (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    valid_time  datetime64[ns] 2018-01-01T06:00:00
Attributes:
    GRIB_paramId:                             130
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      1038240
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    GRIB_gridType:                            regular_ll
    GRIB_NV:                                  0
    GRIB_Nx:                                  1440
    GRIB_Ny:                                  721
    GRIB_cfName:                              air_temperature
    GRIB_cfVarName:                           t
    GRIB_gridDefinitionDescription:           Latitude/longitude. Also called...
    GRIB_iDirectionIncrementInDegrees:        0.25
    GRIB_iScansNegatively:                    0
    GRIB_jDirectionIncrementInDegrees:        0.25
    GRIB_jPointsAreConsecutive:               0
    GRIB_jScansPositively:                    0
    GRIB_latitudeOfFirstGridPointInDegrees:   90.0
    GRIB_latitudeOfLastGridPointInDegrees:    -90.0
    GRIB_longitudeOfFirstGridPointInDegrees:  0.0
    GRIB_longitudeOfLastGridPointInDegrees:   359.75
    GRIB_missingValue:                        9999
    GRIB_name:                                Temperature
    GRIB_shortName:                           t
    GRIB_units:                               K
    long_name:                                Temperature
    units:                                    K
    standard_name:                            air_temperature

In [95]:
ds_t.shape

(721, 1440)

In [96]:
ds_t

<xarray.DataArray 't' (latitude: 721, longitude: 1440)>
[1038240 values with dtype=float32]
Coordinates:
    time        datetime64[ns] 2018-01-01
    step        timedelta64[ns] 06:00:00
    surface     float64 0.0
  * latitude    (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    valid_time  datetime64[ns] 2018-01-01T06:00:00
Attributes:
    GRIB_paramId:                             130
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      1038240
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    GRIB_gridType:                            regular_ll
    GRIB_NV:                                  0
    GRIB_Nx:                                  1440
    GRIB_Ny:                                  721
    GRIB_cfName:                              air_temperature
    GRIB_cfVarName:                           t
    GRIB_gridDefinitionDescription:           Latitude/longitude. Also called...
    GRIB_iDirectionIncrementInDegrees:        0.25
    GRIB_iScansNegatively:                    0
    GRIB_jDirectionIncrementInDegrees:        0.25
    GRIB_jPointsAreConsecutive:               0
    GRIB_jScansPositively:                    0
    GRIB_latitudeOfFirstGridPointInDegrees:   90.0
    GRIB_latitudeOfLastGridPointInDegrees:    -90.0
    GRIB_longitudeOfFirstGridPointInDegrees:  0.0
    GRIB_longitudeOfLastGridPointInDegrees:   359.75
    GRIB_missingValue:                        9999
    GRIB_name:                                Temperature
    GRIB_shortName:                           t
    GRIB_units:                               K
    long_name:                                Temperature
    units:                                    K
    standard_name:                            air_temperature

In [79]:
!pwd

/local/capstone


In [ ]:
### Subset data to different regions by lat / lon boundaries. 

In [165]:
#Define lat/lon bounds of our regions of interest. 

#DUMMY DATA
#Los Angeles
la_min_lat =  89.50
la_max_lat = 90.00
la_min_lon = 0.00
la_max_lon = 0.50

#Tapei 
tp_min_lat =  88.50
tp_max_lat = 89.00
tp_min_lon = 1.00
tp_max_lon = 1.50

#Delhi
dl_min_lat =  87.50
dl_max_lat = 88.00
dl_min_lon = 2.00
dl_max_lon = 2.50

In [166]:
#Filter by appropriate lat/lon bounds
def subset_dataset(dataset, min_lat, max_lat, min_lon, max_lon): 
    '''Takes a dataset and bounding coordinates and returns a filtered subset for the region of interest'''
    mask_lat = np.logical_and(dataset.coords['latitude'] >= min_lat, dataset.coords['latitude'] <= max_lat)
    mask_lon = np.logical_and(dataset.coords['longitude'] >= min_lon, dataset.coords['longitude'] <= max_lon)
    ds_filt = dataset.where(mask_lat & mask_lon, drop = True)
    return ds_filt

In [167]:
ds_t = subset_dataset(ds_t, la_min_lat, la_max_lat, la_min_lon, la_max_lon)
ds_pbl = subset_dataset(ds_pbl, la_min_lat, la_max_lat, la_min_lon, la_max_lon)
ds_r = subset_dataset(ds_r, la_min_lat, la_max_lat, la_min_lon, la_max_lon)

In [168]:
#Make sure we preserve the type of level (atmospheric) of the observation to preserve metadata within the variable names
df_t = ds_t.to_dataframe(name = var_t)
df_t = df_t.drop(columns = ['surface', 'time', 'step'])
df_t = df_t.rename(columns = {"t" : "t_surface", "hpbl" : "pbl_surface", "r" : "r_atmosphere_single_layer"})

df_pbl = ds_pbl.to_dataframe(name = var_hpbl)
df_pbl = df_pbl.drop(columns = ['surface', 'time', 'step'])
df_pbl = df_pbl.rename(columns = {"t" : "t_surface", "hpbl" : "pbl_surface", "r" : "r_atmosphere_single_layer"})

df_r = ds_r.to_dataframe(name = var_r)
df_r = df_r.drop(columns = ['atmosphereSingleLayer', 'time', 'step'])
df_r = df_r.rename(columns = {"t" : "t_surface", "hpbl" : "pbl_surface", "r" : "r_atmosphere_single_layer"})

In [169]:
df_t

valid_time   t_surface
latitude longitude                                
90.00    0.00      2018-01-01 06:00:00  246.139130
         0.25      2018-01-01 06:00:00  246.139130
         0.50      2018-01-01 06:00:00  246.139130
89.75    0.00      2018-01-01 06:00:00  246.439133
         0.25      2018-01-01 06:00:00  246.439133
         0.50      2018-01-01 06:00:00  246.439133
89.50    0.00      2018-01-01 06:00:00  246.439133
         0.25      2018-01-01 06:00:00  246.439133
         0.50      2018-01-01 06:00:00  246.439133

In [170]:
df_pbl

valid_time  pbl_surface
latitude longitude                                 
90.00    0.00      2018-01-01 06:00:00   246.139130
         0.25      2018-01-01 06:00:00   246.139130
         0.50      2018-01-01 06:00:00   246.139130
89.75    0.00      2018-01-01 06:00:00   246.439133
         0.25      2018-01-01 06:00:00   246.439133
         0.50      2018-01-01 06:00:00   246.439133
89.50    0.00      2018-01-01 06:00:00   246.439133
         0.25      2018-01-01 06:00:00   246.439133
         0.50      2018-01-01 06:00:00   246.439133

In [171]:
df_r

valid_time  r_atmosphere_single_layer
latitude longitude                                               
90.00    0.00      2018-01-01 06:00:00                       21.0
         0.25      2018-01-01 06:00:00                       21.0
         0.50      2018-01-01 06:00:00                       21.0
89.75    0.00      2018-01-01 06:00:00                       21.0
         0.25      2018-01-01 06:00:00                       21.0
         0.50      2018-01-01 06:00:00                       21.0
89.50    0.00      2018-01-01 06:00:00                       21.0
         0.25      2018-01-01 06:00:00                       21.0
         0.50      2018-01-01 06:00:00                       21.0

In [172]:
joined_df = pd.merge(df_t, df_pbl, on = ["latitude", "longitude", "valid_time"], how = "left")
joined_df = pd.merge(joined_df, df_r, on = ["latitude", "longitude", "valid_time"], how = "left")
joined_df

valid_time   t_surface  pbl_surface  \
latitude longitude                                                
90.00    0.00      2018-01-01 06:00:00  246.139130   246.139130   
         0.25      2018-01-01 06:00:00  246.139130   246.139130   
         0.50      2018-01-01 06:00:00  246.139130   246.139130   
89.75    0.00      2018-01-01 06:00:00  246.439133   246.439133   
         0.25      2018-01-01 06:00:00  246.439133   246.439133   
         0.50      2018-01-01 06:00:00  246.439133   246.439133   
89.50    0.00      2018-01-01 06:00:00  246.439133   246.439133   
         0.25      2018-01-01 06:00:00  246.439133   246.439133   
         0.50      2018-01-01 06:00:00  246.439133   246.439133   

                    r_atmosphere_single_layer  
latitude longitude                             
90.00    0.00                            21.0  
         0.25                            21.0  
         0.50                            21.0  
89.75    0.00                            21.0  
         0.25                            21.0  
         0.50                            21.0  
89.50    0.00                            21.0  
         0.25                            21.0  
         0.50                            21.0

In [209]:
#Convert to csv, with the appropriate metadata in file name (will extract as field names later). 
filepath = '../train/GFS/' + valid_time_str + '_' + type_of_level + '_' + var_name + '.csv'
xarray_extras.csv.to_csv(x = ds_filt, path = filepath, compression = None)

## Jerico next steps: 
- Confirm t = 'temperature' (email response pending from NCAR w/ full schema)
- Pull in all of Planetary Boundary-Layer Height (PBL), Relative Humidity, Surface Air Temp for 1 file. 
- Before exporting csv: 
    - subset xarray DataArray to relevenat lat/lon bounds 
    - pivot the lat x lon DataArray into a time, (lat,lon), field1, etc. columnar dataset. 
    - Save this columnar, tabular dataset as a csv for a given forecast time and drop the stored grib file from memory. 
    - Iterate to next forecast time grib2 file and repeat until entire date rnage is covered. 
    - Will have ~1095 csv files. These can then be appended into a large DataFrame to join to other datasets (by time, (lat,lon)). 
    